<a href="https://colab.research.google.com/github/UiinKim/Crawling/blob/main/keyword_mapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from google.colab import files

# 파일 업로드
uploaded = files.upload()

# 업로드된 파일 이름 추출
file_name = next(iter(uploaded))

# 'utf-8-sig'로 파일 읽기
df = pd.read_csv(file_name, encoding='cp949')

# 데이터프레임 확인
print(df)

Saving 카테고리_추출.csv to 카테고리_추출 (3).csv
       Unnamed: 0                                             review
0               0  장점 첫 번째로 제가 진짜 피부가 예민해서 화장품을 좀만 잘못 써도 엄청 일어나서 ...
1               1  주변에 쓰는사람들도 많고 가을이 되니 건조함이 느껴져서 미스트가 필요해서 마침 세일...
2               2  달바 미스트 품질이야 뭐 더 말하면 입 아프고요 사은품 처음 받아봤는데 이렇게 풍성...
3               3  이전에 선물받아서 사용했을 때 좋았던 기억이 있어서 구매했어요 요즘 날씨때문에 얼굴...
4               4  향도 좋고 분사하면 광채가 느껴지고 흡수도 빨라요 날이 건조해져서 수시로 사용하려고...
...           ...                                                ...
68318       68318                                               좋아요 
68319       68319                                     좋더라고요 잘 쓰고 있어요
68320       68320                        늘쓰는제품입니다 이만큼 촉촉하고 순한것 못찾겠어요
68321       68321                                       항상 쓰는 건데 좋아요
68322       68322                                         토너입니다 대용량 

[68323 rows x 2 columns]


In [2]:
# 파일 업로드
uploaded = files.upload()

# 업로드된 파일 이름 추출
file_name1 = next(iter(uploaded))

# 'utf-8-sig'로 파일 읽기
test = pd.read_csv(file_name1, encoding='cp949')

# 데이터프레임 확인
print(test)

Saving data.csv to data (2).csv
                                            input
0                    피부가 건조해서 수분이 많은 화장품을 추천해주세요.
1                  지성 피부를 위한 효과적인 지성 화장품을 추천해주세요.
2                  화장품 향에 민감해서 향이 없는 화장품을 추천해주세요.
3           겨울철 건조함에 대비하여 강력한 보습 효과의 화장품을 추천해주세요.
4          피부의 부드러움을 살리기 위해 발림성이 우수한 화장품을 추천해주세요.
..                                            ...
97                 수분이 부족한 지성인데 보습이 좋고 순한 제품을 알려줘
98       평소에 피부가 건조해서 수분 공급이 충분했으면 좋겠고 향이 없으면 좋겠어
99   피부 타입은 모르겠고 평소 여드름이 자주 나서 자극이 없는 순한 제품을 사고싶어
100          속건조가 개선되면 좋겠고 건성에게 적합한 무향의 제품을 찾고 싶어
101              촉촉하면서 자극이 없고 발랐을때 향이 좋은 제품을 추천해줘

[102 rows x 1 columns]


In [3]:
df = df.dropna(subset=['review'])

In [4]:
df=df.drop(['Unnamed: 0'], axis=1)

In [5]:
!pip install transformers scikit-learn pandas


In [ ]:
from transformers import AutoTokenizer, AutoModel
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import torch

# KoBERT 모델과 토크나이저 불러오기
tokenizer = AutoTokenizer.from_pretrained("kykim/bert-kor-base")
model = AutoModel.from_pretrained("kykim/bert-kor-base")

def get_top_similar_reviews(input_text, top_n=3):
    # Tokenize and get embeddings for the input text
    input_tokens = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        input_embedding = model(**input_tokens).last_hidden_state.mean(dim=1).squeeze().numpy()

    # Tokenize and get embeddings for all reviews
    review_tokens = tokenizer(df['review'].tolist(), return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        review_embeddings = model(**review_tokens).last_hidden_state.mean(dim=1).squeeze().numpy()

    # Calculate cosine similarity between input and all reviews
    similarity_scores = cosine_similarity([input_embedding], review_embeddings).flatten()

    # Get indices of top N similar reviews
    top_indices = similarity_scores.argsort()[-top_n:][::-1]

    # Return the top N reviews
    return df.loc[top_indices, 'review']

# 예제 사용법
input_text = [input for input in test]
similar_reviews = get_top_similar_reviews(input_text)
print(similar_reviews)